# Traitement et modélisation des images
<br>

## Preprocessing

#### Importation des ressources

In [1]:
# Importation des librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.transform import resize
from skimage.color import rgb2gray

# Importation du module os pour accéder aux images
import os
path = '../../../images/image_train'
list_img = os.listdir(path)

print('le nombre total d\'image dans le repertoire image_train est de :', len(list_img))

le nombre total d'image dans le repertoire image_train est de : 84916


#### fusion des deux datasets

In [2]:
# Chargement des fichiers X_train et Y_train
X_train = pd.read_csv('../../../X_train.csv', index_col=0)
y_train = pd.read_csv('../../../y_train.csv', index_col=0)

# Fusion avec merge des deux datasets
df = pd.merge(X_train, y_train, left_index = True, right_index = True)

#### Creation d'une nouvelle colonne contenant le chemin pour accéder à l'image

In [3]:
# création d'une nouvelle colonne
df['Nom image'] = ['image_' + str(imageid) + '_product_' + str(productid) + '.jpg' for imageid, productid in zip(df['imageid'], df['productid'])]
df['lien'] = str(path) + '/' + df['Nom image']

#### Prétraitement des images 

In [4]:
# Fonction pour lire et prétraiter une image
def preprocess_image(image_path, target_size=(64, 64)):
    image = imread(image_path)
    image = rgb2gray(image)  # Convertir en noir et blanc
    image = resize(image, target_size)  # Redimensionner l'image
    return image.flatten()  # Aplatir l'image en un vecteur 1D

# Appliquer le prétraitement à toutes les images et stocker les caractéristiques
features = []
for image_lien in df['lien']:
    features.append(preprocess_image(image_lien))

features = np.array(features)

## Modelisation Random Forest

#### Importation des ressources

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import joblib

#### Entrainement

In [11]:
# Préparer les données pour l'entraînement
X = features
y = df['prdtypecode'].values

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuration de GridSearchCV
"""
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 7],
    'min_samples_leaf': [1, 2, 3]
}
"""
param_grid = {
    'n_estimators': [50, 100, 150]
}

# Initialiser le modèle Random Forest
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Initialiser GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

# Exécuter GridSearchCV sur les données d'entraînement
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres
print("Meilleurs paramètres:", grid_search.best_params_)

# Utiliser le meilleur modèle trouvé par GridSearchCV
best_model = grid_search.best_estimator_

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Meilleurs paramètres: {'n_estimators': 150}


#### Evaluation du modèle 

In [12]:
# Évaluer le modèle
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy du meilleur modèle: {accuracy*100:.2f}%')

Accuracy du meilleur modèle: 47.76%


#### Sauvegarde du model

In [13]:
# Sauvegarder le meilleur modèle
joblib.dump(best_model, 'random_forest_best_model.joblib')

['random_forest_best_model.joblib']

## Modelisation Deep Learning

#### Importation des ressources

In [16]:
# Importation des librairies nécessaires de Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_tuner import RandomSearch, HyperParameters


#### Ajustement des datas pour le CNN

In [19]:
# Conversion de 'prdtypecode' en chaîne de caractère
df['prdtypecode'] = df['prdtypecode'].astype(str)

In [21]:
# Configuration de ImageDataGenerator pour le prétraitement et l'augmentation des données
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% des données pour la validation
)

# Générateur pour l'ensemble d'entraînement et de validation
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,  # Aucun répertoire car les chemins sont complets dans `df['lien']`
    x_col='lien',  # La colonne contenant les chemins complets des images
    y_col='prdtypecode',  # La colonne contenant les étiquettes
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=None,
    x_col='lien',
    y_col='prdtypecode',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Fonction pour construire le modèle avec optimisation des hyperparamètres
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('kernel_size', values=[3, 5]),
        activation='relu',
        input_shape=(64, 64, 3)
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    for i in range(hp.Int('conv_layers', 1, 3)):  # Boucle ajoutant des couches Conv2D supplémentaires
        model.add(Conv2D(
            filters=hp.Int(f'filters_{i+2}', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice(f'kernel_size_{i+2}', values=[3, 5]),
            activation='relu'
        ))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(
        units=hp.Int('units', min_value=32, max_value=512, step=32),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(27, activation='softmax'))
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Configuration et démarrage de la recherche d'hyperparamètres comme GridSearchCV 
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory="./keras_tuner_dir",
    project_name='keras_tuner_demo'
)

tuner.search(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=3)]
)

# Récupération du meilleur modèle
best_model = tuner.get_best_models(num_models=1)[0]
best_model.evaluate(validation_generator)


Trial 20 Complete [04h 53m 35s]
val_accuracy: 0.47476887702941895

Best val_accuracy So Far: 0.47476887702941895
Total elapsed time: 1d 19h 31m 47s
531/531 [==============================] - 183s 344ms/step - loss: 1.8879 - accuracy: 0.4748


[1.88787043094635, 0.47476887702941895]

##### Sauvegarde

In [22]:
# Sauvegarde du modèle au format HDF5
best_model.save('cnn_images_model.h5')

# Sauvegarde du modèle au format SavedModel (le format par défaut)
best_model.save('cnn_images_model')

# Sauvegarde uniquement des poids du modèle
best_model.save_weights('cnn_images_weights.h5')


c:\Users\tgp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: cnn_images_model\assets


INFO:tensorflow:Assets written to: cnn_images_model\assets


In [23]:
# Sauvegarde du modèle au format Keras natif
best_model.save('cnn_images_model.keras')


##### Results

In [26]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:\n")

for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")

best_model_metrics = best_model.evaluate(validation_generator)
print("\nResultat du Best Model:")
print(f"Loss: {best_model_metrics[0]}, Accuracy: {best_model_metrics[1]}")

# Comparaison avec les meilleurs 5 autres combinaisons de paramètres
all_trials = tuner.oracle.get_best_trials(num_trials=5)

print("\nTop Trials:")
for trial in all_trials:
    print(f"Trial {trial.trial_id}, Accuracy: {trial.score}, Parameters: {trial.hyperparameters.values}")


Best Hyperparameters:

filters: 96
kernel_size: 5
conv_layers: 2
filters_2: 96
kernel_size_2: 3
units: 416
dropout: 0.30000000000000004
learning_rate: 0.0001
filters_3: 96
kernel_size_3: 5
filters_4: 64
kernel_size_4: 3
531/531 [==============================] - 149s 281ms/step - loss: 1.8879 - accuracy: 0.4748

Resultat du Best Model:
Loss: 1.8878705501556396, Accuracy: 0.47476887702941895

Top Trials:
Trial 19, Accuracy: 0.47476887702941895, Parameters: {'filters': 96, 'kernel_size': 5, 'conv_layers': 2, 'filters_2': 96, 'kernel_size_2': 3, 'units': 416, 'dropout': 0.30000000000000004, 'learning_rate': 0.0001, 'filters_3': 96, 'kernel_size_3': 5, 'filters_4': 64, 'kernel_size_4': 3}
Trial 17, Accuracy: 0.46469998359680176, Parameters: {'filters': 64, 'kernel_size': 5, 'conv_layers': 2, 'filters_2': 128, 'kernel_size_2': 3, 'units': 448, 'dropout': 0.2, 'learning_rate': 0.0001, 'filters_3': 64, 'kernel_size_3': 5, 'filters_4': 64, 'kernel_size_4': 3}
Trial 13, Accuracy: 0.458222925662

##### Fine-tuning

In [27]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.00001,
    verbose=1
)

history = best_model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Nouvelle evaluation après ré-entrainement
final_metrics = best_model.evaluate(validation_generator)
print(f"Performance finale: Loss: {final_metrics[0]}, Accuracy: {final_metrics[1]}")


Epoch 1/50
2123/2123 [==============================] - 1658s 781ms/step - loss: 1.3168 - accuracy: 0.6043 - val_loss: 1.8960 - val_accuracy: 0.4765 - lr: 1.0000e-04
Epoch 2/50
2123/2123 [==============================] - 1493s 703ms/step - loss: 1.2388 - accuracy: 0.6280 - val_loss: 1.9268 - val_accuracy: 0.4725 - lr: 1.0000e-04
Epoch 3/50
2123/2123 [==============================] - 1465s 690ms/step - loss: 1.1608 - accuracy: 0.6456 - val_loss: 1.9644 - val_accuracy: 0.4735 - lr: 1.0000e-04
Epoch 4/50
2123/2123 [==============================] - 1448s 682ms/step - loss: 1.0866 - accuracy: 0.6654 - val_loss: 2.0168 - val_accuracy: 0.4715 - lr: 1.0000e-04
Epoch 5/50
2123/2123 [==============================] - 1036s 488ms/step - loss: 1.0156 - accuracy: 0.6874 - val_loss: 2.0663 - val_accuracy: 0.4775 - lr: 1.0000e-04
Epoch 6/50
2123/2123 [==============================] - ETA: 0s - loss: 0.9498 - accuracy: 0.7063
Epoch 6: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e

##### Sauvegarde après fine-tuning

In [28]:
# Sauvegarde du modèle au format HDF5
best_model.save('cnn_images_model_improved.h5')

# Sauvegarde du modèle au format SavedModel (le format par défaut)
best_model.save('cnn_images_model_improved')

# Sauvegarde uniquement des poids du modèle
best_model.save_weights('cnn_images_improved_weights.h5')

best_model.save('cnn_images_model_improved.keras')

c:\Users\tgp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: cnn_images_model_improved\assets


INFO:tensorflow:Assets written to: cnn_images_model_improved\assets
